In [ ]:
# Imports
%matplotlib inline

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab.patches import cv2_imshow
from IPython.display import Image

img1 = cv.imread('data/frame10.png', cv.IMREAD_UNCHANGED)
#cv2_imshow(img1)

img2 = cv.imread('data/frame11.png', cv.IMREAD_UNCHANGED)
#cv2_imshow(img2)

Image(open('data/Dimetrodon.gif','rb').read())

Corner / Feature Detector

In [ ]:
# Parameters for Shi Tomasi features
feature_params = dict(maxCorners=0,  # no limit on number of corners
                      qualityLevel=0.15,
                      minDistance=50,
                      blockSize=7)

def shi_tomasi(img):
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  return cv.goodFeaturesToTrack(gray, **feature_params)

In [ ]:
corners1 = shi_tomasi(img1)
img1_pts = np.copy(img1)
corners1 = np.int0(corners1)
for i in corners1:
    x,y = i.ravel()
    cv.circle(img1_pts,(x,y),3,255,-1)

cv2_imshow(img1_pts)

Optical Flow - Lukas Kanade

In [ ]:
# Parameters for Lucas Kanade optical flow
optical_flow_params = dict(winSize=(15, 15),
                           maxLevel=2, 
                           criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

def optical_flow(frame0, frame1, corners0):        
    # convert images to grayscale
    frame0_gray = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
    frame1_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

    # Indices of the `status` array which equal 1 signify a corresponding new feature has been found
    corners1, status, err = cv.calcOpticalFlowPyrLK(frame0_gray, frame1_gray, corners0, None, **optical_flow_params)
    
    return corners1, status==1

In [ ]:
corners1 = np.float32(corners1)
corners2, valid = optical_flow(img1, img2, corners1)
velocity = ((corners2 - corners1)[valid==1]).reshape(-1, 2)

# Draw OF lines on image
c = 0
corners1 = np.int0(corners1)
img1_LK = np.copy(img1)
for i in corners1:
    x_start = i.ravel()[0]
    y_start = i.ravel()[1]
    x_end = x_start + np.int0(velocity[c][0])
    y_end = y_start + np.int0(velocity[c][1])
    cv.arrowedLine(img1_LK,(x_start, y_start),(x_end, y_end),(255,0,0),1)
    c += 1

cv2_imshow(img1_LK)

In [ ]:
# Zoom into 1 OF vector
img1_LK_zoom = cv.resize(img1_LK[0:50, 300:350], None, fx=5, fy=5)
cv2_imshow(img1_LK_zoom)

In [ ]:
optical_flow_params = dict(winSize=(5, 5),
                           maxLevel=5, 
                           criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

###Coarse-To-Fine method
img_LK_start = cv.imread('data/LK_start.jpg', cv.IMREAD_UNCHANGED)
img_LK_disp = cv.imread('data/LK_small_disp.jpg', cv.IMREAD_UNCHANGED)
#img_LK_disp = cv.imread('data/LK_large_disp.jpg', cv.IMREAD_UNCHANGED)

# Get features
corners1 = shi_tomasi(img_LK_start)

corners1 = np.float32(corners1)
corners2, valid = optical_flow(img_LK_start, img_LK_disp, corners1)
velocity = ((corners2 - corners1)[valid==1]).reshape(-1, 2)

# Draw OF lines on image
c = 0
corners1 = np.int0(corners1)
img_shape_LK = np.copy(img_LK_start)
for i in corners1:
    x_start = i.ravel()[0]
    y_start = i.ravel()[1]
    x_end = x_start + np.int0(velocity[c][0])
    y_end = y_start + np.int0(velocity[c][1])
    cv.arrowedLine(img_shape_LK,(x_start, y_start),(x_end, y_end),(255,0,0),1)
    c += 1

print(img_shape_LK.shape)
cv2_imshow(img_shape_LK)

img_shape_LK_disp = np.copy(img_LK_disp)
c = 0
for i in corners1:
    x_start = i.ravel()[0]
    y_start = i.ravel()[1]
    x_end = x_start + np.int0(velocity[c][0])
    y_end = y_start + np.int0(velocity[c][1])
    cv.arrowedLine(img_shape_LK_disp,(x_start, y_start),(x_end, y_end),(255,0,0),1)
    c += 1

#cv2_imshow(img_shape_LK_disp)
cv2_imshow(img_shape_LK_disp)

Optical Flow - Gunnar Farneback

In [ ]:
# Create mask
hsv_mask = np.zeros_like(img1)
# Make image saturation to a maximum value
hsv_mask[..., 1] = 255

frame1_gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
frame2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
gf_flow = cv.calcOpticalFlowFarneback(frame1_gray, frame2_gray, None, 0.5, 3, 8, 5, 5, 1.1, cv.OPTFLOW_FARNEBACK_GAUSSIAN)

# Compute magnite and angle of 2D vector
mag, ang = cv.cartToPolar(gf_flow[..., 0], gf_flow[..., 1])
# Set image hue value according to the angle of optical flow
hsv_mask[..., 0] = ang * 180 / np.pi / 2
# Set value as per the normalized magnitude of optical flow
hsv_mask[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
# Convert to rgb
rgb_rep = cv.cvtColor(hsv_mask, cv.COLOR_HSV2BGR)

cv2_imshow(rgb_rep)

Optical Flow - TVL1

In [ ]:
# Create mask
hsv_mask_tvl1 = np.zeros_like(img1)
# Make image saturation to a maximum value
hsv_mask_tvl1[..., 1] = 255

tvl1 = cv.optflow.createOptFlow_DualTVL1()
tvl1_flow = tvl1.calc(frame1_gray, frame2_gray, None)

# Compute magnite and angle of 2D vector
mag, ang = cv.cartToPolar(tvl1_flow[..., 0], tvl1_flow[..., 1])
# Set image hue value according to the angle of optical flow
hsv_mask_tvl1[..., 0] = ang * 180 / np.pi / 2
# Set value as per the normalized magnitude of optical flow
hsv_mask_tvl1[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
# Convert to rgb
rgb_rep_tvl1 = cv.cvtColor(hsv_mask_tvl1, cv.COLOR_HSV2BGR)

cv2_imshow(rgb_rep_tvl1)

Grabbing From Video

In [ ]:
import PIL
import io
from base64 import b64decode, b64encode

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
from IPython.display import Javascript
from google.colab.output import eval_js

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
prev_0 = cv.imread('frame10.png', cv.IMREAD_UNCHANGED)
prev = cv.resize(prev_0, (640,480))
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    next = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # LK-Method start
    # corners1 = shi_tomasi(prev)
    # corners1 = np.int0(corners1)

    # for i in corners1:
    #   x,y = i.ravel()
    #   cv.circle(bbox_array,(x,y),3,255,-1)

    # corners1 = np.float32(corners1)
    # corners2, valid = optical_flow(prev, next, corners1)
    # velocity = ((corners2 - corners1)[valid==1]).reshape(-1, 2)

    # c = 0
    # corners1 = np.int0(corners1)
    # for i in ((corners1)[valid==1]):
    #   x_start = i.ravel()[0]
    #   y_start = i.ravel()[1]
    #   x_end = x_start + np.int0(velocity[c][0])
    #   y_end = y_start + np.int0(velocity[c][1])
    #   cv.line(bbox_array,(x_start, y_start),(x_end, y_end),(255,0,0),1)
    #   c += 1
    # LK-Method end

    # Gunner Farneback start
    # hsv_mask = np.zeros_like(prev)
    # # Make image saturation to a maximum value
    # hsv_mask[..., 1] = 255

    # frame1_gray = cv.cvtColor(prev, cv.COLOR_BGR2GRAY)
    # frame2_gray = cv.cvtColor(next, cv.COLOR_BGR2GRAY)
    # gf_flow = cv.calcOpticalFlowFarneback(frame1_gray, frame2_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # # Compute magnite and angle of 2D vector
    # mag, ang = cv.cartToPolar(gf_flow[..., 0], gf_flow[..., 1])
    # # Set image hue value according to the angle of optical flow
    # hsv_mask[..., 0] = ang * 180 / np.pi / 2
    # # Set value as per the normalized magnitude of optical flow
    # hsv_mask[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    # # Convert to rgb
    # rgb_rep = cv.cvtColor(hsv_mask, cv.COLOR_HSV2BGR)
    # rgb_repA = cv.cvtColor(hsv_mask, cv.COLOR_BGR2BGRA)
    # bbox_array = np.copy(rgb_repA)
    # Gunner Farneback end

    # Compute magnite and angle of 2D vector
    mag, ang = cv.cartToPolar(tvl1_flow[..., 0], tvl1_flow[..., 1])
    # Set image hue value according to the angle of optical flow
    hsv_mask_tvl1[..., 0] = ang * 180 / np.pi / 2
    # Set value as per the normalized magnitude of optical flow
    hsv_mask_tvl1[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    # Convert to rgb
    rgb_rep_tvl1 = cv.cvtColor(hsv_mask_tvl1, cv.COLOR_HSV2BGR)
    rgb_rep_tvl1A = cv.cvtColor(hsv_mask_tvl1, cv.COLOR_BGR2BGRA)
    bbox_array = np.copy(rgb_rep_tvl1A)
    # TVL1 end

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    prev = np.copy(next)